In [1]:
import numpy as np
from random import choices
from random import randint
from random import random
import random
import typing
from typing import List
from typing import Tuple
from collections import defaultdict
import matplotlib.pyplot as plt
import pdb
import matplotlib
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# from scipy.optimize import differential_evolution
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [ ]:
bootstrap_lookup = {1: True, 2: False}
max_depth_lookup = np.arange(1, 100)
max_features_lookup = {1: 'auto', 2: 'sqrt', 3: 'log2'}
min_samples_leaf_lookup = np.arange(1,5)
min_samples_split_lookup = np.arange(2,10)
n_estimators_lookup = np.arange(1,2000)
constraints = [ bootstrap_lookup, max_depth_lookup, max_features_lookup,  min_samples_leaf_lookup, min_samples_split_lookup, n_estimators_lookup ]

In [159]:
bootstrap_lookup, max_depth_lookup, max_features_lookup, min_samples_leaf_lookup, min_samples_split_lookup, n_estimators_lookup = [{1: True, 2: False}, np.arange(1, 100), {1: 'auto', 2: 'sqrt', 3: 'log2'}, np.arange(1,5), np.arange(2,10), np.arange(1,2000)]


In [12]:
bootstrap_lookup = {1: True, 2: False}
max_depth_lookup = np.arange(1, 100)
max_features_lookup = {1: 'auto', 2: 'sqrt', 3: 'log2'}
min_samples_leaf_lookup = np.arange(1,5)
min_samples_split_lookup = np.arange(2,10)
n_estimators_lookup = np.arange(2000)
constraints = [ bootstrap_lookup, max_depth_lookup, max_features_lookup, min_samples_leaf_lookup, min_samples_split_lookup, n_estimators_lookup ]

In [161]:
org_df['Event'].value_counts(normalize=True)

False    0.9984
True     0.0016
Name: Event, dtype: float64

In [160]:
org_df['Event'].value_counts(normalize=True)

False    132268
True        212
Name: Event, dtype: int64

In [3]:
org_df = pd.read_csv('waterDataTraining.csv')
df = org_df.copy()
df.dropna(inplace = True)
df.head()

,Time,Tp,pH,Cond,Turb,SAC,PFM,Event
0,2017-07-01 00:00:00,6.94,8.60774,0.020954,0.125931,3.58683,43.7559,False
1,2017-07-01 00:01:00,6.93,8.60589,0.020965,0.127219,3.59025,43.4366,False
2,2017-07-01 00:02:00,6.94,8.60220,0.020968,0.126482,3.58318,43.5994,False
3,2017-07-01 00:03:00,6.94,8.60220,0.020972,0.126184,3.58769,43.3704,False
4,2017-07-01 00:04:00,6.94,8.60405,0.020973,0.127908,3.58287,43.1656,False


In [4]:
X = df.drop(['Time', 'Event'], axis = 1)
y = df['Event']

In [5]:
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

 X_train dataset:  (92548, 6)
 y_train dataset:  (92548,)
 X_test dataset:  (39664, 6)
 y_test dataset:  (39664,)


In [6]:
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

In [13]:
from sklearn.ensemble import RandomForestClassifier

In [14]:
def generate_population(n, constraints):
    population = np.array([])
    for i in constraints:
        if type(i) == dict:#
            value = list(i.keys())
        else:
            value = i
        options = np.array(choices(k = n, population = value)).reshape(-1,1)
        if len(population) == 0:
            population = options
        else:
            population = np.hstack((population, options))
    return population

In [15]:
def calc_fitness(solution, constraints):
    bootstrap = bootstrap_lookup[solution[0]]
    max_depth = solution[1]
    max_features = max_features_lookup[solution[2]]
    min_samples_leaf = solution[3]
    min_samples_split = solution[4]
    n_estimators = solution[5]
    model = RandomForestClassifier(bootstrap = bootstrap, max_depth = max_depth,
                                  max_features = max_features, min_samples_leaf = min_samples_leaf,
                                  min_samples_split = min_samples_split, n_estimators = n_estimators)
    model.fit(X_train_res, y_train_res)
    y_pred = model.predict(X_test)
    metrics = get_metrics(y_test, y_pred)
    return metrics

In [17]:
from sklearn.metrics import confusion_matrix
def get_metrics(Y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(Y_true, y_pred).ravel()
    acc = (tp + tn) / (tp + fp + tn + fn)
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    f1 = 2 * (precision * recall) / (precision + recall)
    return acc, precision, recall, f1

In [20]:
def single_point_crossover(parent_1, parent_2):
    assert len(list(parent_1)) == len(list(parent_2)), 'Parents should be of same length'
    crossover_point = randint(1, len(parent_1)) -1
    child_1 = np.hstack((parent_1[0:crossover_point], parent_2[crossover_point:]))
    child_2 = np.hstack((parent_2[0:crossover_point], parent_1[crossover_point:]))
    return child_1, child_2

In [21]:
def mutation(solution, k):
    solution = list(solution.copy())
    locus = 0
    for i in np.arange(0,k):
        locus = randint(1, len(solution)) - 1
        parameter = constraints[locus]
        if type(parameter) == dict:#
            value = list(parameter.keys())
        else:
            value = parameter
        new_value = np.array(choices(k = 1, population = value))[0]
        solution[locus] =new_value
    return solution

In [132]:
def weakest_replacement(fitness_list, pop, child):
    fitness_f1_tracker = [f1 for acc, prec, recall, f1 in fitness_list]
    existing_worst_fitness = min(fitness_f1_tracker)
    current_weakest_positions = np.where(fitness_f1_tracker == existing_worst_fitness)[0]
    target = choices(current_weakest_positions, k = 1)[0]
    fitness_of_child = calc_fitness(child, constraints)
    if fitness_of_child[3] > existing_worst_fitness:
        print('yes, replacing', existing_worst_fitness, fitness_of_child[3],)
        pop[target] = child
        fitness_list[target] = fitness_of_child
    return pop, fitness_list

In [23]:
def binary_tournament(pop):
    return np.array(choices(population = list(pop),k=2))

In [34]:
population = generate_population(10, constraints = constraints)
fitness_tracker = []
for solution in population:
    score = calc_fitness(solution, constraints)
    fitness_tracker.append(score)

In [130]:
# fitness_tracker = act_fitness_tracker.copy()
# population = act_pop.copy()

In [62]:
# act_pop = population.copy()
# act_fitness_tracker = fitness_tracker.copy()

In [155]:
generations_fitness = []
for i in range(5):
    p1, p2 = binary_tournament(population)
    c1, c2 = p1,p2
    c1, c2 = single_point_crossover(p1, p2)
    c1 = mutation(c1, k = 1)
    c2 = mutation(c2, k = 1)
    population, fitness_tracker = weakest_replacement(child = c1, pop = population, fitness_list = fitness_tracker)    
    population, fitness_tracker = weakest_replacement(child = c2, pop = population, fitness_list = fitness_tracker)
    generations_fitness.append(np.array(fitness_tracker))

[(0.9996218233158531, 0.9444444444444444, 0.8095238095238095, 0.8717948717948718), (0.99959661153691, 0.9122807017543859, 0.8253968253968254, 0.8666666666666667), (0.9996470350947962, 0.9298245614035088, 0.8412698412698413, 0.8833333333333334), (0.99959661153691, 0.8852459016393442, 0.8571428571428571, 0.8709677419354839), (0.9996218233158531, 0.9285714285714286, 0.8253968253968254, 0.8739495798319328), (0.99959661153691, 0.8983050847457628, 0.8412698412698413, 0.8688524590163935), (0.9996722468737395, 0.9464285714285714, 0.8412698412698413, 0.8907563025210083), (0.9996218233158531, 0.9285714285714286, 0.8253968253968254, 0.8739495798319328), (0.9996218233158531, 0.9285714285714286, 0.8253968253968254, 0.8739495798319328), (0.9996218233158531, 0.9285714285714286, 0.8253968253968254, 0.8739495798319328)]
yes, replacing 0.8666666666666667 0.8717948717948718
yes, replacing 0.8688524590163935 0.8717948717948718
[(0.9996218233158531, 0.9444444444444444, 0.8095238095238095, 0.871794871794871

In [156]:
abcd = [(0.9996218233158531, 0.9285714285714286, 0.8253968253968254, 0.8739495798319328), (0.9996470350947962, 0.9298245614035088, 0.8412698412698413, 0.8833333333333334), (0.9996470350947962, 0.9298245614035088, 0.8412698412698413, 0.8833333333333334), (0.9996470350947962, 0.9298245614035088, 0.8412698412698413, 0.8833333333333334), (0.9996218233158531, 0.9285714285714286, 0.8253968253968254, 0.8739495798319328), (0.9996218233158531, 0.9137931034482759, 0.8412698412698413, 0.8760330578512397), (0.9996722468737395, 0.9464285714285714, 0.8412698412698413, 0.8907563025210083), (0.9996218233158531, 0.9285714285714286, 0.8253968253968254, 0.8739495798319328), (0.9996218233158531, 0.9137931034482759, 0.8412698412698413, 0.8760330578512397), (0.9996218233158531, 0.9285714285714286, 0.8253968253968254, 0.8739495798319328)]
last = []
for a, b, c ,d in abcd:
    last.append(d)

In [148]:
abcd = [(0.9996218233158531, 0.9444444444444444, 0.8095238095238095, 0.8717948717948718), (0.99959661153691, 0.9433962264150944, 0.7936507936507936, 0.8620689655172413), (0.9995461879790238, 0.8688524590163934, 0.8412698412698413, 0.8548387096774194), (0.99959661153691, 0.8852459016393442, 0.8571428571428571, 0.8709677419354839), (0.9996218233158531, 0.9285714285714286, 0.8253968253968254, 0.8739495798319328), (0.99959661153691, 0.8983050847457628, 0.8412698412698413, 0.8688524590163935), (0.9996722468737395, 0.9464285714285714, 0.8412698412698413, 0.8907563025210083), (0.99959661153691, 0.9433962264150944, 0.7936507936507936, 0.8620689655172413), (0.9995713997579669, 0.8833333333333333, 0.8412698412698413, 0.8617886178861788), (0.9996218233158531, 0.9285714285714286, 0.8253968253968254, 0.8739495798319328)]
first = []
for a, b, c ,d in abcd:
    first.append(d)

In [157]:
min(last)

0.8739495798319328

In [150]:
min(first)

0.8548387096774194

In [151]:
max(first)

0.8907563025210083

In [158]:
max(last)

0.8907563025210083

In [153]:
np.mean(first)

0.8691035793529703

In [154]:
np.mean(last)

0.8748169694595489

In [42]:
[f1 for acc, prec, recall, f1 in fitness_tracker]

[0.8717948717948718,
 0.8620689655172413,
 0.8548387096774194,
 0.8709677419354839,
 0.8739495798319328,
 0.8688524590163935,
 0.8907563025210083,
 0.8620689655172413,
 0.8617886178861788,
 0.8739495798319328]

In [56]:
arr = []
min_fitness = ''
for i in generations_fitness:
    k = 0
    for j in i:
        print(j[3])
        k+=1
    print('\n')

0.8717948717948718
0.8620689655172413
0.8548387096774194
0.8709677419354839
0.8739495798319328
0.8688524590163935
0.8907563025210083
0.8620689655172413
0.8617886178861788
0.8739495798319328


0.8717948717948718
0.8620689655172413
0.8548387096774194
0.8709677419354839
0.8739495798319328
0.8688524590163935
0.8907563025210083
0.8620689655172413
0.8617886178861788
0.8739495798319328


0.8717948717948718
0.8620689655172413
0.8548387096774194
0.8709677419354839
0.8739495798319328
0.8688524590163935
0.8907563025210083
0.8620689655172413
0.8617886178861788
0.8739495798319328


0.8717948717948718
0.8620689655172413
0.8548387096774194
0.8709677419354839
0.8739495798319328
0.8688524590163935
0.8907563025210083
0.8620689655172413
0.8617886178861788
0.8739495798319328


0.8717948717948718
0.8620689655172413
0.8548387096774194
0.8709677419354839
0.8739495798319328
0.8688524590163935
0.8907563025210083
0.8620689655172413
0.8617886178861788
0.8739495798319328




In [57]:
for i in generations_fitness:
    for j in i:
        print(j)

(0.9996218233158531, 0.9444444444444444, 0.8095238095238095, 0.8717948717948718)
(0.99959661153691, 0.9433962264150944, 0.7936507936507936, 0.8620689655172413)
(0.9995461879790238, 0.8688524590163934, 0.8412698412698413, 0.8548387096774194)
(0.99959661153691, 0.8852459016393442, 0.8571428571428571, 0.8709677419354839)
(0.9996218233158531, 0.9285714285714286, 0.8253968253968254, 0.8739495798319328)
(0.99959661153691, 0.8983050847457628, 0.8412698412698413, 0.8688524590163935)
(0.9996722468737395, 0.9464285714285714, 0.8412698412698413, 0.8907563025210083)
(0.99959661153691, 0.9433962264150944, 0.7936507936507936, 0.8620689655172413)
(0.9995713997579669, 0.8833333333333333, 0.8412698412698413, 0.8617886178861788)
(0.9996218233158531, 0.9285714285714286, 0.8253968253968254, 0.8739495798319328)
(0.9996218233158531, 0.9444444444444444, 0.8095238095238095, 0.8717948717948718)
(0.99959661153691, 0.9433962264150944, 0.7936507936507936, 0.8620689655172413)
(0.9995461879790238, 0.868852459016393

In [ ]:
arr = []
min_fitness = ''
for i in generations_fitness:
    k = 0
    for j in i:
        if min_fitness =='':
            min_fitness = j[3]
        else:
            if j[3] < min_fitness:
                min_fitness = j[3]
                

In [43]:
g1 = np.array(generations_fitness[0])
g1[:, 3]

array([0.87179487, 0.86206897, 0.85483871, 0.87096774, 0.87394958,
       0.86885246, 0.8907563 , 0.86206897, 0.86178862, 0.87394958])

In [47]:
gen_c = 0
for gen in generations_fitness:
    print('Gen', {gen_c})
    gen_c+=1
    for pop_fit in gen:
        for acc, prec, recall, f1 in gen:
            print(f1)


Gen {0}
0.8717948717948718
0.8620689655172413
0.8548387096774194
0.8709677419354839
0.8739495798319328
0.8688524590163935
0.8907563025210083
0.8620689655172413
0.8617886178861788
0.8739495798319328
0.8717948717948718
0.8620689655172413
0.8548387096774194
0.8709677419354839
0.8739495798319328
0.8688524590163935
0.8907563025210083
0.8620689655172413
0.8617886178861788
0.8739495798319328
0.8717948717948718
0.8620689655172413
0.8548387096774194
0.8709677419354839
0.8739495798319328
0.8688524590163935
0.8907563025210083
0.8620689655172413
0.8617886178861788
0.8739495798319328
0.8717948717948718
0.8620689655172413
0.8548387096774194
0.8709677419354839
0.8739495798319328
0.8688524590163935
0.8907563025210083
0.8620689655172413
0.8617886178861788
0.8739495798319328
0.8717948717948718
0.8620689655172413
0.8548387096774194
0.8709677419354839
0.8739495798319328
0.8688524590163935
0.8907563025210083
0.8620689655172413
0.8617886178861788
0.8739495798319328
0.8717948717948718
0.8620689655172413
0.85

In [516]:
for i in generations_fitness:
    for acc, prec, recall, f1 in i:
        print(f1)

0.8620689655172413
0.8760330578512397
0.8688524590163935
0.8833333333333334
0.8617886178861788
0.8780487804878048
0.864406779661017
0.8717948717948718
0.8833333333333334
0.8666666666666667
0.8620689655172413
0.8760330578512397
0.8688524590163935
0.8833333333333334
0.8617886178861788
0.8780487804878048
0.864406779661017
0.8717948717948718
0.8833333333333334
0.8666666666666667
0.8620689655172413
0.8760330578512397
0.8688524590163935
0.8833333333333334
0.8617886178861788
0.8780487804878048
0.864406779661017
0.8717948717948718
0.8833333333333334
0.8666666666666667
0.8620689655172413
0.8760330578512397
0.8688524590163935
0.8833333333333334
0.8617886178861788
0.8780487804878048
0.864406779661017
0.8717948717948718
0.8833333333333334
0.8666666666666667
0.8620689655172413
0.8760330578512397
0.8688524590163935
0.8833333333333334
0.8617886178861788
0.8780487804878048
0.864406779661017
0.8717948717948718
0.8833333333333334
0.8666666666666667


In [504]:
for act_fitness_tracker in generations_fitness:
#     print(act_fitness_tracker)
     print(max(act_fitness_tracker))

(0.9996470350947962, 0.9298245614035088, 0.8412698412698413, 0.8833333333333334)
(0.9996470350947962, 0.9298245614035088, 0.8412698412698413, 0.8833333333333334)
(0.9996470350947962, 0.9298245614035088, 0.8412698412698413, 0.8833333333333334)
(0.9996470350947962, 0.9298245614035088, 0.8412698412698413, 0.8833333333333334)
(0.9996470350947962, 0.9298245614035088, 0.8412698412698413, 0.8833333333333334)
